# Exploratory Data Analysis and Pre-processing

In this module you will learn how to load an online retail dataset in Python, visualise and summarise it to produce insights that will guide your machine learning endevours in the next modules. 

You will learn how to plot data and calculate summary statistics to build a dataset from which you ultimately will try to predict future customer behaviour.

For this module, you will need to import 

* `numpy` (as `np`)  [link to documentation](https://docs.scipy.org/doc/)
* `pandas` (as `pd`) [link to documentation](https://pandas.pydata.org/pandas-docs/stable/)
* `matplotlib.pyplot` (as `plt`) [link to documentation](https://matplotlib.org/contents.html)
* `seaborn` (as `sns`) [link to documentation](https://seaborn.pydata.org)


In order to use `matplotlib` in the notebook, recall that you need to have a cell with
```python
%matplotlib inline
```

In [1]:
# add your code here to load the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


### Loading the dataset


Load the dataset `online_retail_customer_data.csv` with `pandas`. Recall that you need to 

* use the `read_csv()` method from `pandas`
* point to the location of the dataset
* determine a name under which you want to store the resulting data frame (we suggest the name `customers`)
* specify that the `CustomerID` column is the index column using the `index_col` option

Use the `head` method to display the first few lines of the dataset (you can specify how many lines).

In [2]:
customers = pd.read_csv("data/online_retail_customer_data.csv", 
                       index_col = "CustomerID")
customers.head()

,Country,balance,max_spent,mean_spent,min_spent,n_orders,time_between_orders,total_items,total_items_returned,total_refunded,total_spent
CustomerID,,,,,,,,,,,
12346,United Kingdom,0.00,77183.6,38591.800000,0.0,2.0,NaN,74215.0,-74215.0,-77183.60,77183.60
12348,Finland,3874.60,2248.8,1291.533333,478.8,3.0,54.500000,6876.0,0.0,0.00,3874.60
12350,Norway,294.40,294.4,294.400000,294.4,1.0,NaN,196.0,0.0,0.00,294.40
12352,Norway,1845.13,1054.1,393.092000,0.0,5.0,11.333333,774.0,-63.0,-120.33,1965.46
12354,Spain,1079.40,1079.4,1079.400000,1079.4,1.0,NaN,530.0,0.0,0.00,1079.40



### About the dataset

The dataset is based on data from an online retailer selling gifts and is based on a dataset taken from [here](https://archive.ics.uci.edu/ml/datasets/Online+Retail#).

We have taken the original data set and processed it to create a 'profile' for each customer, which includes a number of features such as:

* `Country`: The country the purchases were made from.
* `balance`: Amount of money spent at the store (purchases minus refunds).
* `n_orders`: Total number of orders from the online retailer.
* `time_between_orders`: Average time (in days) between orders.
* `max_spent`: Most amount of money customer spent on a single order.

Check the dimensionality of the dataset using the `shape` attribute of the data frame.

In [3]:
# add your code to check the dimensions of the dataset
customers.shape

(3254, 11)

## Exploring the dataset

At this stage you do not really know what is going on in this dataset. 
You need to go beyond the first impression by considering simple questions like:

* How many customers are you dealing with?
* What country spends how much?
* What has been the company's profit during the last year?
* Are there differences between customers that return and those who don't?

You will go through these questions and learn new tricks as you move along. The first one is easy to answer, you can use the `nunique()` method applied to the index column. You can retrieve the index column using the `index` attribute of the dataframe.

In [4]:
# use the nunique method
customers.index.nunique()


3254

That could have been expected!!

### Types of features and One-Hot-Encoding

Data can be both 

* on a *continuous* scale: e.g.: the amount of money spent in the store or time between purchases
* on a *discrete* scale: e.g.: the country.

Discrete variables that have a notion of ordering (for instance, a survey that asks your satisfaction from scale of one to five) are called *ordinal*. Discrete variables that cannot be ordered are usually referred to as *categorical* variables (eg: countries or gender).

In the feature engineering step, one typically needs to pay special attention to discrete variables as many models are not equipped to handle this type of data, particularly if they are just categorical.

In the case where categorical features are present, you need to represent them as numerical values. 
A standard approach to do so is the **one-hot encoding**. 
The input in one-hot encoding is the vector of discrete categorical values, and the output is a sparse matrix where each column corresponds to one possible value of the feature.
As an example, let's consider the following trivial dataset

```
Nick, UK
Laura, IT
Massimo, IT
```

In this case, there are two countries `["UK", "IT"]`, the one-hot-encoding would correspond to the table

$$
\begin{array}{l|cc}
& \text{UK} & \text{IT} \\\hline
\text{Nick} & 1 & 0\\
\text{Laura} &0& 1\\
\text{Massimo}&0 & 1
\end{array}
$$

In order to do that on the original dataset:

* Use the function `get_dummies()` from `pandas` on the series `Country` from `customers`
* Apply `set_index(customers.index)` to the corresponding object so it has the same index as our original dataframe. This is so that you'll be able to join them later on. (*Can you explain what this command does?*)
* Save the new dataframe in a variable `countries`
* use `head()` to have a look and make sure it all makes sense


In [12]:
countries = pd.get_dummies(customers.Country).set_index(customers.index)
countries.head()

,Austria,Belgium,Finland,France,Germany,Italy,Norway,Portugal,Spain,Switzerland,United Kingdom
CustomerID,,,,,,,,,,,
12346,0,0,0,0,0,0,0,0,0,0,1
12348,0,0,1,0,0,0,0,0,0,0,0
12350,0,0,0,0,0,0,1,0,0,0,0
12352,0,0,0,0,0,0,1,0,0,0,0
12354,0,0,0,0,0,0,0,0,1,0,0


You can now drop the original `Country` column from the `customers` dataframe and keep this one. For this, use `pandas`' `drop()` function. Don't forget to use the `inplace` parameter if you want your change to be appied on the dataframe.

In [13]:
customers.drop('Country', axis = 1, inplace=True)

We are going to do pre-processing and imputation on the continuous features, so let's keep this separate for now and join it together later.

### Missing values and Imputation

You may have noticed that there are missing values in the data (`NaN`).
It's very important in general to check whether there are any and

* whether these missing values are informative or not
* whether you can replace the missing values in a sensible way or not

First, check which column has missing values and how many.
For this, use

* the `isnull()` method applied on the data frame, this returns a dataframe similar to the original one but where every entry is just `True` or `False`
* on the resulting dataframe, apply the `sum()` method which will count how many entries of the column are `True`

In [16]:
customers.isnull().sum()

balance                    0
max_spent                  0
mean_spent                 0
min_spent                  0
n_orders                   0
time_between_orders     1368
total_items                0
total_items_returned       0
total_refunded             0
total_spent                0
dtype: int64

*What do you think might be the reason why some customers have missing values as their time between orders?*

In general, for columns with missing values, there are a few choices on how to handle them. 
This process is usually called *imputation*.

#### Imputation 

There are many strategies to help with missing data and they depend on whether the missing data is numeric or categorical. Recall that you can for example

* simply remove rows where there is missing data (e.g. `.dropna()` can achieve this)
* imputing the values with a summary statistic such as mean or median or most frequent value (e.g. `Imputer` from `sklearn` module)
* replace the values with a sensible estimate

What strategy is best for you problem very much depends on the specifics of your dataset. 

In the current case, the missing values are exclusively found in the `time_between_orders` column, so you should have a look at these rows where this occurs to see if we can gain an understanding of what may be causing these missing values.

* select the customers for which `time_between_orders` is null. For this, use `isnull` on the appropriate column and feed it as row indices to the dataframe to retrieve a subdataframe only corresponding to those customers
* check the shape, make sure it worked!

In [21]:
# add your code here to find the instances where time_between_orders is empty
msk_null_time = customers.time_between_orders.isnull()
null_time = customers[msk_null_time]

In [22]:
# how many nan cases do you have? have a look
null_time.shape

(1368, 10)

Note that `n_orders` seems to often be equal to `1`. 
There is a fairly obvious interpretation for those, since they haven't yet come back, there is no "time between orders".
You can count the number of time specific values of `n_orders` occur by using the `value_counts()` method applied on the series corresponding to `n_orders`. 

In [23]:
null_time.n_orders.value_counts()

1.0    1168
2.0     175
3.0      21
4.0       3
8.0       1
Name: n_orders, dtype: int64

The majority of missing values can therefore be explained by customers that have not returned before.
It is unclear at this point what the remaining 200 are.

This dataset is of course fairly artificial so we won't discuss this in too much detail at this point. 

In this case, we decide to fill the missing values by 365 days by using the `fillna()` function (this is a fairly reckless decision but, again, this notebook is more focused on tools and techniques)

* replace the column `time_between_orders` by the same column where the missing values are filled with value 365 using the `fillna()` function applied on the column
* use `head()` to check

In [24]:
customers.time_between_orders.fillna(value=365, inplace=True)
customers.head()


,balance,max_spent,mean_spent,min_spent,n_orders,time_between_orders,total_items,total_items_returned,total_refunded,total_spent
CustomerID,,,,,,,,,,
12346,0.00,77183.6,38591.800000,0.0,2.0,365.000000,74215.0,-74215.0,-77183.60,77183.60
12348,3874.60,2248.8,1291.533333,478.8,3.0,54.500000,6876.0,0.0,0.00,3874.60
12350,294.40,294.4,294.400000,294.4,1.0,365.000000,196.0,0.0,0.00,294.40
12352,1845.13,1054.1,393.092000,0.0,5.0,11.333333,774.0,-63.0,-120.33,1965.46
12354,1079.40,1079.4,1079.400000,1079.4,1.0,365.000000,530.0,0.0,0.00,1079.40


### Removing Outliers

Outliers are observation that appear extreme relative to the bulk of the data.
Machine Learning techniques can be sensitive to outliers. 

Here you will see how you can get rid of them if that's what you decide to do. 
There are multiple ways to define outliers, one possibility is to consider all points that are more than `k` standard deviations (`sigma`, $\sigma$) away from the mean (`mu`, $\mu$) of the data.

Below you can see a simple function that takes data and a number of standard deviations and filters out everything that doesn't lie in the range $[\mu-k\sigma, \mu+k\sigma]$.

In [25]:
# this function takes a Series and filters out all elements that are outside
# the range [mu-k*sigma , mu+k*sigma]
def remove_outliers(data, k=3):
    mu       = data.mean() # get the mean
    sigma    = data.std()  # get the standard deviation
    filtered = data[(mu-k*sigma < data) & (data < mu+k*sigma)]
    return filtered

You can `.apply()` this function on your dataframe. It will call it on each column individually and will return a new dataframe where values declared as outliers are replaced by `NaN`, keeping the structure of the `pd.DataFrame` intact.

In [26]:
customers = customers.apply(remove_outliers)


Let's now remove the lines with `NaN` values (that correspond to lines with outliers). For this, use the `dropna()` method on the dataframe. 

In [27]:
customers.dropna(inplace=True)
customers.shape


(3126, 10)

### Scaling

The different numerical variables have completely different scales. This becomes even more obvious when considering a boxplot. You'll use the `seaborn` wrapper around `matplotlib` that is great for producing clear plots.
Have a look [here](https://stanford.edu/~mwaskom/software/seaborn/examples/index.html) for a gallery of plots possible with `seaborn`.

* define a figure environment with the `figure()` method of `matplotlib.pyplot` (you can pass a figure size)
* use the `boxplot` function of `seaborn` specifying the appropriate dataframe

In [8]:
# add your code to plot a sns.boxplot() of the customer dataframe


You can see that `n_orders` is defined in a much narrower space than `balance`. 
If you were to use the data in an unscaled form, the effect of `balance` might be disproportionnaly high and cause a Machine Learning algorithm to underperform. 

To account for this, it is good practice to scale your data, so that all the dimensions fall onto a comparable interval. To do this:

* Define a "scaler" using the `StandardScaler` class imported from `sklearn.preprocessing` (you could also use the `MinMaxScaler`)
* Apply it on the dataframe using the `fit_transform` method 
* Define a new dataframe similar to the original one but with scaled columns

In [9]:
# add your code here to apply the scaling


Let's replot the `boxplot`.

In [10]:
# add your code here to replot the boxplot with the scaled data


## Relationship between input features

An important tool for the exploratory data analysis step is the **scatter plot**. 

This plot helps visualise the relationship in-between two input features. It may also give you a first indication of the Machine Learning model that could be applied and its complexity (linear vs. non-linear). 

Create a scatter plot of the `n_orders` vs `balance` using the `lmplot` function of `seaborn`. Once you have a grip of this, you should try looking at the scatter plot corresponding to other couples.

### (Bonus) - Grid/scatterplot matrix

A scatterplot matrix shows a grid of all scatterplots where each attribute is plotted against all other attributes.
This can be applied when there aren't too many variables (otherwise it quickly becomes impractical). 

You can find further information on how to create a scatterplot matrix with seaborn using the `pairplot()` function [here](https://stanford.edu/~mwaskom/software/seaborn/generated/seaborn.pairplot.html).


### Correlation matrix and heatmap of correlations between the input features

It is often of great interest to investigate whether any of the variables in a multivariate dataset are significantly correlated. 
As previously shown, the different features (variables) in `customers` are not independent from each other. 
To quickly identify which features are related and to what degree, it is useful to compute a correlation matrix that shows the correlation coefficient for each pair of variables. 
You can do this by using the `corr()` function from the `pandas` library:

To visualise the degree of correlation between variables, you can use a heatmap (also from `seaborn`). 

### Variance thresholding

Variance thresholding is a simple approach that removes features with low variance, in order to reduce dimensionality while preserving as much of the variation expressed by the data as possible/practical.

Importantly, this thresholding does not take any classification into account, so we are examining the variance for a given feature across samples, not the variance relative to any output or class.

Variance thresholding is implemented as a transformer object in `scikit-learn` with a number of different options.

* Join the `countries` dataframe corresponding to the one-hot encoding to the `customers` dataframe
* create an object of the `VarianceThreshold` class from `sklearn.feature_selection` to select the subset of features with variance of at least `0.5`
* run the `fit()` method on the object, you can then use the `get_support()` method to see an array of True/False for which columns pass the threshold

In [11]:
# add your code here for the variance thresholding 


*Which features would be removed based on this strategy?
What are the drawbacks of this method?*